In [2]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "../04-advanced-conv/cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

/home/luke/.virtualenvs/tf1.13_py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/luke/.virtualenvs/tf1.13_py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/luke/.virtualenvs/tf1.13_py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/luke/.virtualenvs/tf1.13_py3/lib/python3

['data_batch_1', 'data_batch_5', 'data_batch_2', 'test_batch', 'batches.meta', 'readme.html', 'data_batch_4', 'data_batch_3']


In [3]:
# tensorboard
# 1. 指定面板图上显示的变量
# 2. 训练过程中将这些变量计算出来，输出到文件中
# 3. 文件解析 ./tensorboard --logdir=dir.  在ubuntu内部的terminal启动

In [4]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.Dataset.
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [5]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# conv1: 神经元图， feature_map, 输出图像
conv1_1 = tf.layers.conv2d(x_image,
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv1_1')
conv1_2 = tf.layers.conv2d(conv1_1,
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv1_2')
# 16 * 16
pooling1 = tf.layers.max_pooling2d(conv1_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool1')

conv2_1 = tf.layers.conv2d(pooling1,
                           32,  # output channel number
                           (3,3),  # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv2_1')
conv2_2 = tf.layers.conv2d(conv2_1,
                           32,  # output channel number
                           (3,3),  # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv2_2')
# 8 * 8
pooling2 = tf.layers.max_pooling2d(conv2_2,
                                   (2, 2),  # kernel size
                                   (2, 2),  # stride
                                   name = 'pool2')

conv3_1 = tf.layers.conv2d(pooling2,
                           32,  # output channel number
                           (3,3),  # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv3_1')
conv3_2 = tf.layers.conv2d(conv3_1,
                           32,  # output channel number
                           (3,3),  # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv3_2')
# 4 * 4 * 32
pooling3 = tf.layers.max_pooling2d(conv3_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool3')
# [None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [6]:
!rm -rf run_vgg_tensorboard

# 相对于之前的vgg主要是增加下面一个单元格内容

In [7]:
# 该单元格的主要作用是输出对应的数据，从而来绘制图形，相当于多输出一些
def variable_summary(var, name):
    """Constructs summary for statistics of a variable"""
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)  # 均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))  # 标准差
        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.histogram('histogram', var)  # 直接把变量放进去看直方图

with tf.name_scope('summary'):  # 下面的merge_all会把我们这里的都放进去
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')

loss_summary = tf.summary.scalar('loss', loss)  # loss是上面一个单元格中的结点
# 'loss': <10, 1.1>, <20, 1.08> tf.summary.scalar会进行一个聚合，就是把每一次训练的值进行记录
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

source_image = (x_image + 1) * 127.5  # 做一个归一化的逆过程
inputs_summary = tf.summary.image('inputs_image', source_image)

merged_summary = tf.summary.merge_all()  # 训练集关注 输入（输入的方方面面）、loss，accuracy,
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary])  # 测试集只关注loss,accuracy

# 下面是创建文件夹，这里是和2.0创建文件夹类似
LOG_DIR = '.'
run_label = './run_vgg_tensorboard'
run_dir = os.path.join(LOG_DIR, run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')
if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

# 下面的源码增加了对merged_summary和merged_summary_test的计算

In [8]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100
output_summary_every_steps = 100  # 每100次计算一次summary


# train 10000: 70.45%
with tf.Session() as sess:
    sess.run(init)
    # 训练集合测试集均建立一个句柄，和python写文件类似
    train_writer = tf.summary.FileWriter(train_log_dir, sess.graph) 
    test_writer = tf.summary.FileWriter(test_log_dir)
    
    fixed_test_batch_data, fixed_test_batch_labels \
        = test_data.next_batch(batch_size)

    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        eval_ops = [loss, accuracy, train_op]
         # 每100次去计算一次summary
        should_output_summary = ((i+1) % output_summary_every_steps == 0)
        if should_output_summary:
            eval_ops.append(merged_summary)
    
        eval_ops_results = sess.run(
            eval_ops,
            feed_dict={
                x: batch_data,
                y: batch_labels})
        loss_val, acc_val = eval_ops_results[0:2]  # eval_ops_results和eval_ops位置对应的
        if should_output_summary:
            # 有4个时（分别是loss, accuracy, train_op，merged_summary），是最后一个，因此eval_ops_results[-1] 
            train_summary_str = eval_ops_results[-1] 
            train_writer.add_summary(train_summary_str, i+1)  # 写入文件
            test_summary_str = sess.run([merged_summary_test],
                                        feed_dict={
                                            x: fixed_test_batch_data,  # 需要固定测试输入和测试标签
                                            y: fixed_test_batch_labels,
                                        })[0]
            test_writer.add_summary(test_summary_str, i+1)
        
        # 下面代码和之前没有区别
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step: 100, loss: 2.19051, acc: 0.20000
[Train] Step: 200, loss: 2.04229, acc: 0.30000
[Train] Step: 300, loss: 1.91441, acc: 0.35000
[Train] Step: 400, loss: 1.34035, acc: 0.60000
[Train] Step: 500, loss: 1.84559, acc: 0.55000
[Train] Step: 600, loss: 1.75535, acc: 0.30000
[Train] Step: 700, loss: 1.75523, acc: 0.30000
[Train] Step: 800, loss: 2.07712, acc: 0.40000
[Train] Step: 900, loss: 1.14683, acc: 0.65000
[Train] Step: 1000, loss: 1.62998, acc: 0.45000
(10000, 3072)
(10000,)
[Test ] Step: 1000, acc: 0.51500
[Train] Step: 1100, loss: 1.31388, acc: 0.60000
[Train] Step: 1200, loss: 1.68664, acc: 0.45000
[Train] Step: 1300, loss: 1.15098, acc: 0.50000
[Train] Step: 1400, loss: 1.13936, acc: 0.55000
[Train] Step: 1500, loss: 1.17392, acc: 0.60000
[Train] Step: 1600, loss: 1.03068, acc: 0.70000
[Train] Step: 1700, loss: 1.50034, acc: 0.55000
[Train] Step: 1800, loss: 1.56837, acc: 0.50000
[Train] Step: 1900, loss: 1.41793, acc: 0.40000
[Train] Step: 2000, loss: 1.87106, acc: 0